# Explore models
Try SOTA text summarization models for their performance on speech/lecture transcripts
Since Sat. Oct. 23rd, 2021


In [1]:
## Setup
import torch
# import tensorflow
from transformers import pipeline
from transformers import AutoModelWithLMHead, AutoTokenizer
from transformers import PegasusTokenizer, BigBirdPegasusForConditionalGeneration, BigBirdPegasusConfig

from util import *


txt_ted = get_ted_eg()
ic(len(get_1st_n_words(txt_ted, as_arr=True)))
txt_498 = get_498_eg()
ic(len(get_1st_n_words(txt_498, as_arr=True)))



ic| len(get_1st_n_words(txt_ted, as_arr=True)): 3130
ic| len(get_1st_n_words(txt_498, as_arr=True)): 10548


10548

## BigBird
"Big Bird: Transformers for Longer Sequences".
Manzil Zaheer, Guru Guruganesh, Kumar Avinava Dubey, Joshua Ainslie, Chris Alberti, Santiago Ontanon, Philip Pham, Anirudh Ravula, Qifan Wang, Li Yang, Amr Ahmed.
*NeurIPS 2020*.



In [2]:
def bigbird(text):
    """
    `BigBird` in torch

    Looks like doesn't support summarization
    """
    summarizer = pipeline(
        'summarization',
        model='google/bigbird-roberta-base',
        tokenizer='google/bigbird-roberta-base'
    )
    summarizer(text, min_length=5, max_length=20)
# bigbird('hello world')


def bigbird_pegasus(text):
    model = BigBirdPegasusForConditionalGeneration.from_pretrained('google/bigbird-pegasus-large-arxiv')
    tokenizer = PegasusTokenizer.from_pretrained('google/bigbird-pegasus-large-arxiv')
    inputs = tokenizer([text], max_length=4096 * 2, return_tensors='pt', truncation=True)
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=256, early_stopping=True)
    return tokenizer.batch_decode(summary_ids)
"""
Hyper-parameter seems to affect the result drastically

On TED.com `Do schools kill creativity?`
'<s> we present a brief discussion of the nature of education in the era of '
'big data and big rip.<n> we start with a brief history of education in the '
'era of big rip.<n> we then turn to a brief discussion of the nature of '
'education in the era of big rip.<n> we conclude with a discussion of the '
'future of education.'
"""


ic(txt_498[:400])
# txt = get_1st_n_words(txt)
# ic(txt)
ic(bigbird_pegasus(txt_ted))
ic(bigbird_pegasus(txt_498))


ic| txt_498[:400]: ("No, you don't need a reason. Apple TV. All right. Apple TV. Sweet don't have "
                    "to carry around a dongle. Welcome, welcome everybody. Yeah, it's um the "
                    "gloomy day. I got a bunch of great emails saying I can't make it. I feel it. "
                    "I feel it. But anyway, let's just jump right into how many people have "
                    "groups? Awesome. That's all you left. The scene. Pretty much everybody. So "
                    "we'll will tease out who mad")
/Users/stefanh/Documents/UMich/Development/Team Sirious/Sirious-Assistant/venv/lib/python3.9/site-packages/transformers/models/bigbird_pegasus/modeling_bigbird_pegasus.py:790: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, roundin

KeyboardInterrupt: 

## T5



In [3]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")


def summarize(text, max_length=150):
    input_ids = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True)
    generated_ids = model.generate(input_ids=input_ids, num_beams=2, max_length=max_length,  repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)
    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
    return preds[0]


text = get_ted_eg()
ic(summarize(text))



Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

ImportError: 
T5Converter requires the protobuf library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/protocolbuffers/protobuf/tree/master/python#installation and follow the ones
that match your environment.
